In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time : ${2023-6-27} ${21:05}
# @Author : Mathew Jin
# @File : ${run_model.py}
# chmod +x run_model.py
# python3 scripts/run_model3.py --process=0 --rawname=1 --resume=0 --anchoring=0
'''
1.TRA
https://arxiv.org/pdf/2106.12950.pdf
https://github.com/microsoft/qlib/blob/main/examples/benchmarks/TRA/src/model.py
1.1 HIST
https://arxiv.org/pdf/2110.13716.pdf
https://github.com/Wentao-Xu/HIST
2.Lightgbm
https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/plot_example.py
https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.plot_tree.html
3.other factors
'''
import argparse
import torch
import torch.nn as nn
import numpy as np
import itertools , random , os, shutil , gc , time , h5py , yaml
from copy import deepcopy
from torch.optim.swa_utils import AveragedModel , update_bn
from scripts.util.environ import get_logger
from scripts.util.basic import process_timer , FilteredIterator , lr_cosine_scheduler , versatile_storage
from scripts.util.multiloss import multiloss_calculator 
from scripts.util.trainer import trainer_parser , train_config , set_trainer_environment , Device
from scripts.data_util.ModelData import ModelData
from scripts.function.basic import *
from scripts.nn.My import *
# from audtorch.metrics.functional import *

try:
    parser = trainer_parser().parse_args()
except:
    parser = trainer_parser().parse_args(args=[])

logger = get_logger()
config = train_config(parser = parser , do_process=True)
set_trainer_environment(config)
time_recorder = process_timer(True)
storage_model = versatile_storage(config.storage_type)

/home/mengkjin/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
usage: ipykernel_launcher.py [-h] [--process PROCESS] [--rawname RAWNAME]
                             [--resume RESUME] [--anchoring ANCHORING]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/mengkjin/.local/share/jupyter/runtime/kernel-v2-489767zR7DxsNNqaBB.json


--What process would you want to run? 0: all, 1: train only (default), 2: test only , 3: copy to instance
--Process Queue : Data + Train + Test + Instance
--Start Training New!
--Model_name is set to LSTM_day_SHORTTEST!


In [19]:
np.nan_to_num([np.nan])

array([0.])

In [15]:
import numpy as np
np.concatenate([np.empty((0,10)),np.random.rand(10,10)])

array([[0.34580856, 0.6812599 , 0.80064276, 0.73788133, 0.35166539,
        0.61607931, 0.44166507, 0.86284776, 0.5444348 , 0.22884268],
       [0.52830971, 0.87296038, 0.59623307, 0.63503991, 0.87288609,
        0.7604093 , 0.84850689, 0.09116341, 0.95884051, 0.47505979],
       [0.1042597 , 0.49692085, 0.66953094, 0.15427043, 0.42289969,
        0.66122607, 0.19171118, 0.56999261, 0.81849671, 0.52397418],
       [0.77675723, 0.90236865, 0.10116521, 0.63026562, 0.56897266,
        0.94560416, 0.49609061, 0.12047259, 0.56318735, 0.51044792],
       [0.04882031, 0.6890926 , 0.15426646, 0.18384061, 0.56069865,
        0.96371715, 0.73366521, 0.67817439, 0.4430654 , 0.8172947 ],
       [0.58841908, 0.93269105, 0.49017386, 0.49916485, 0.65827105,
        0.09918233, 0.94792628, 0.41251688, 0.06213669, 0.63426524],
       [0.57077226, 0.92319001, 0.44619464, 0.90637105, 0.70659258,
        0.27449223, 0.18813138, 0.61462964, 0.69652102, 0.61450676],
       [0.57621798, 0.86990698, 0.3397029

In [11]:
np.empty((0,0))

array([], shape=(0, 0), dtype=float64)

In [2]:
config.process_queue

['data', 'train', 'test', 'instance']

In [50]:
config.train_params['criterion']['score']

{'loss': 'ccc',
 'score': 'pearson',
 'penalty': {'hidden_orthogonality': {'lamb': 0.001},
  'tra_ot_penalty': {'lamb': 0.01, 'rho': 0.999}}}

In [15]:
import time
a = time.ctime()
time.time() - time.mktime(time.strptime(a))
b = time.time()
time.ctime()

'Sun Jan 28 10:20:14 2024'

In [47]:
import itertools
class FilteredIterator:
    def __init__(self, iterable, condition):
        self.iterable  = iter(iterable)
        if callable(condition):
            self.condition = condition
        else:
            self.condition = iter(condition)
    def __iter__(self):
        return self
    def __next__(self):
        while True:
            item = next(self.iterable)
            cond = self.condition(item) if callable(self.condition) else next(self.condition)
            if cond: return item
model_iter = list(itertools.product(range(3) , range(2)))
models_trained = np.array([True,False,False,False,False,False])
for i,(model_date,model_num) in enumerate(model_iter):
    print(i,model_date,model_num)
model_iter = FilteredIterator(model_iter, models_trained == 0)
for i,x in enumerate(model_iter):
    print(i,x)

0 0 0
1 0 1
2 1 0
3 1 1
4 2 0
5 2 1
0 (0, 1)
1 (1, 0)
2 (1, 1)
3 (2, 0)
4 (2, 1)


In [40]:
list(itertools.product(range(3) , range(2))) , models_trained == 0

([(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)],
 array([False,  True,  True,  True,  True,  True]))

In [28]:
list(itertools.product(range(3) , range(2)))

[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]

In [8]:
getattr(config , 'TRAIN_PARAM' , None)

{'dataloader': {'random_seed': None,
  'random_tv_split': True,
  'sample_method': 'train_shuffle',
  'train_ratio': 0.8},
 'trainer': {'optimizer': {'name': 'Adam', 'param': {}},
  'scheduler': {'name': 'cycle',
   'param': {'base_lr': 1e-07, 'step_size_up': 4}},
  'learn_rate': {'base': 0.005,
   'ratio': {'attempt': [1, 0.1, 10, 0.01, 100],
    'round': [1.0],
    'transfer': 0.1},
   'reset': {'num_reset': 2,
    'trigger': 40,
    'recover_level': 1.0,
    'speedup2x': True}},
  'nanloss': {'retry': 2},
  'gradient': {'clip_value': 10.0},
  'retrain': {'attempts': 4, 'min_epoch': 20, 'min_epoch_round': 10}},
 'criterion': {'loss': 'ccc',
  'score': 'pearson',
  'penalty': {'hidden_orthogonality': {'lamb': 0.001},
   'tra_ot_penalty': {'lamb': 0.01, 'rho': 0.999}}},
 'transfer': False,
 'output_types': ['best', 'swalast', 'swabest'],
 'multitask': {'type': 'hybrid',
  'param_dict': {'dwa': {'tau': 2},
   'ruw': {'phi': None},
   'ewa': {},
   'gls': {},
   'rws': {},
   'hybrid': {